# Tutorial Part 1: Executing Static Ensembles

The RADICAL-EnsembleToolkit (RE) is, obviously, targeting ensemble use cases.  The simplest case is the execution of a _'static'_ ensemple.  _'Static'_ here refers to:

  - the ensemble has a well defined and imutable set of ensemble members;
  - each ensemble member has a well defined and immutable structure;
  - the ensemble is completely specified before its execution starts.
  
RE provides the following API abstractions to specify an ensemple:

  - `re.Pipeline`: ensemble members are rendered as pipelines.
  - `re.Stage`: ensemble pipelines consist of _consecutive_ stages of execution.
  - `re.Task`: pipeline stages consist of _independent_ (and thus concurrent) tasks.
  
The execution of ensembles is managed by an `re.ApplicationManager` instance.


This first tutorial example demonstrates the execution of a simple ensemble of simulations.  Each ensemble member is in itself a pipeline of three different stages:

  1) generate a random seed as input data
  2) evolve a model based on that input data via a set of ensembles
  3) derive a common metric across the model results

Similar patterns are frequently found in MD simulation workflows.  For the purpose of this tutorial, the stages are:

  1) random seed  : create a random number)
  2) evolve model : N tasks computing n'th power of the input)
  3) common metric: sum over all 'model' outputs

The final results are then staged back and printed on STDOUT.


## RE program structure

Load the RE module and create the main program structure:

  - create an `ApplicationManager` instance
  - assign a suitable set of resources to execute the pipeline on
  - assign the ensemble to be executed to the `ApplicationManager`
  - execute the pipeline by calling `ApplicationManager.run()` 

In [1]:
import radical.entk as re


# ------------------------------------------------------------------------------
# 
def main():
    
    # create an application manager which executes our ensemble
    appman = re.AppManager()

    # assign resource request description to the application manager using
    # three mandatory keys: target resource, walltime, and number of cpus
    appman.resource_desc = {
        'resource': 'local.localhost',
      # 'resource': 'local.localhost_flux',
        'walltime': 10,
        'cpus'    : 1
    }

    # create an ensemble of n simulation pipelines
    n_pipelines = 10
    ensemble = set()
    for cnt in range(n_pipelines):
        ensemble.add(generate_pipeline(uid='pipe.%03d' % cnt))

    # assign the workflow to the application manager, then
    # run the ensemble and wait for completion
    appman.workflow = ensemble
    appman.run()

    # check results which were staged back
    for cnt in range(n_pipelines):
        data   = open('pipe.%03d.sqrt.txt' % cnt).read()
        result = float(data)
        print('%3d -- %25.2f' % (cnt, result))

### Ensemble pipelines

In line 23 above we generated the ensemble we expect to execute: a set of `10` pipelines.  We now define the method `generate_pipeline(uid)` to create the indivdual pipelines, as shown below:


In [2]:
# ------------------------------------------------------------------------------
#
def generate_pipeline(uid):
    '''
    Generate a single simulation pipeline, i.e., a new ensemble member.
    The pipeline structure consisting of three steps as described above.
    '''

    # all tasks in this pipeline share the same sandbox
    sandbox = uid

    # first stage: create 1 task to generate a random seed number
    t1 = re.Task()
    t1.executable = '/bin/sh'
    t1.arguments  = ['-c', 'od -An -N1 -i /dev/random']
    t1.stdout     = 'random.txt'
    t1.sandbox    = sandbox

    s1 = re.Stage()
    s1.add_tasks(t1)

    # second stage: create 10 tasks to compute the n'th power of that number
    s2 = re.Stage()
    n_simulations = 10
    for i in range(n_simulations):
        t2 = re.Task()
        t2.executable = '/bin/sh'
        t2.arguments  = ['-c', 'echo "$(cat random.txt) ^ %d" | bc' % i]
        t2.stdout     = 'power.%03d.txt' % i
        t2.sandbox    = sandbox
        s2.add_tasks(t2)

    # third stage: compute sum over all powers
    t3 = re.Task()
    t3.executable = '/bin/sh'
    t3.arguments  = ['-c', 'cat power.*.txt | paste -sd+ | bc']
    t3.stdout     = 'sum.txt'
    t3.sandbox    = sandbox

    s3 = re.Stage()
    s3.add_tasks(t3)

    # fourth stage: compute square root of previous sum
    t4 = re.Task()
    t4.executable = '/bin/sh'
    t4.arguments  = ['-c', 'echo "sqrt($(cat sum.txt))" | bc']
    t4.stdout     = 'sqrt.txt'
    t4.sandbox    = sandbox

    # download the result while renaming to get unique files per pipeline
    t4.download_output_data = ['sqrt.txt > %s.sqrt.txt' % uid]

    s4 = re.Stage()
    s4.add_tasks(t4)

    # assemble the three stages into a pipeline and return it
    p = re.Pipeline()
    p.add_stages(s1)
    p.add_stages(s2)
    p.add_stages(s3)
    p.add_stages(s4)

    return p

### Execution

The `appmgr.run()` method in `main()` is blocking until the ensemble execution completed - we thus just call main and then return.


In [3]:
# ------------------------------------------------------------------------------
#
if __name__ == '__main__':

    main()
    
    
# ------------------------------------------------------------------------------

EnTK session: re.session.rivendell.merzky.018957.0002
Creating AppManagerSetting up RabbitMQ system                                 ok
                                                                              ok
Validating and assigning resource manager                                     ok
Setting up RabbitMQ system                                                   n/a
new session: [re.session.rivendell.merzky.018957.0002]                         \
database   : [mongodb://localhost/am]                                         ok
create pilot manager                                                          ok
submit 1 pilot(s)
        pilot.0000   local.localhost           1 cores       0 gpus           ok
All components created
create task managerUpdate: pipeline.0006 state: SCHEDULING
Update: pipeline.0006.stage.0024 state: SCHEDULING
Update: pipeline.0006.stage.0024.task.0078 state: SCHEDULING
Update: pipeline.0000 state: SCHEDULING
Update: pipeline.0000.stage.0000 state: SCHEDU

Update: pipeline.0006.stage.0025.task.0083 state: SCHEDULING
Update: pipeline.0006.stage.0025.task.0084 state: SCHEDULING
Update: pipeline.0006.stage.0025.task.0085 state: SCHEDULING
Update: pipeline.0006.stage.0025.task.0086 state: SCHEDULING
Update: pipeline.0006.stage.0025.task.0087 state: SCHEDULING
Update: pipeline.0006.stage.0025.task.0079 state: SCHEDULING
Update: pipeline.0006.stage.0025.task.0088 state: SCHEDULING
Update: pipeline.0006.stage.0025.task.0080 state: SCHEDULING
Update: pipeline.0006.stage.0025.task.0081 state: SCHEDULING
Update: pipeline.0000.stage.0001 state: SCHEDULING
Update: pipeline.0000.stage.0001.task.0004 state: SCHEDULING
Update: pipeline.0000.stage.0001.task.0005 state: SCHEDULING
Update: pipeline.0000.stage.0001.task.0007 state: SCHEDULING
Update: pipeline.0000.stage.0001.task.0006 state: SCHEDULING
Update: pipeline.0000.stage.0001.task.0001 state: SCHEDULING
Update: pipeline.0000.stage.0001.task.0002 state: SCHEDULING
Update: pipeline.0000.stage.0001.t

Update: Update: pipeline.0006.stage.0025.task.0083 state: SUBMITTING
pipeline.0000.stage.0001.task.0004 state: SCHEDULED
Update: Update: pipeline.0006.stage.0025.task.0084 state: SUBMITTING
pipeline.0000.stage.0001.task.0005 state: SCHEDULED
submit: Update: Update: pipeline.0006.stage.0025.task.0085 state: SUBMITTING
pipeline.0000.stage.0001.task.0007 state: SCHEDULED
Update: #Update: pipeline.0000.stage.0001.task.0006 state: SCHEDULED
pipeline.0006.stage.0025.task.0086 state: SUBMITTING
Update: pipeline.0000.stage.0001.task.0001 state: SCHEDULED
Update: #pipeline.0006.stage.0025.task.0087 state: SUBMITTING
Update: pipeline.0000.stage.0001.task.0002 state: SCHEDULED
Update: Update: #pipeline.0000.stage.0001.task.0009 state: SCHEDULED
pipeline.0006.stage.0025.task.0079 state: SUBMITTING
Update: Update: pipeline.0006.stage.0025.task.0088 state: SUBMITTING
#pipeline.0000.stage.0001.task.0003 state: SCHEDULED
Update: Update: pipeline.0006.stage.0025.task.0080 state: SUBMITTING
pipeline.000

pipeline.0003.stage.0013.task.0047 state: SCHEDULED
Update: #pipeline.0008.stage.0033.task.0105 state: SUBMITTING
Update: pipeline.0009.stage.0037.task.0126 state: SCHEDULED
Update: Update: #pipeline.0008.stage.0033.task.0106 state: SUBMITTING
pipeline.0009.stage.0037.task.0127 state: SCHEDULED
Update: Update: pipeline.0009.stage.0037.task.0118 state: SCHEDULED
#pipeline.0008.stage.0033.task.0107 state: SUBMITTING
Update: pipeline.0009.stage.0037.task.0119 state: SCHEDULED
Update: #pipeline.0008.stage.0033.task.0108 state: SUBMITTING
Update: pipeline.0009.stage.0037.task.0120 state: SCHEDULED
Update: pipeline.0008.stage.0033.task.0109 state: SUBMITTING
Update: #pipeline.0009.stage.0037.task.0121 state: SCHEDULED
Update: Update: pipeline.0009.stage.0037.task.0122 state: SCHEDULED
pipeline.0008.stage.0033.task.0110 state: SUBMITTING
#Update: Update: pipeline.0008.stage.0033.task.0111 state: SUBMITTING
pipeline.0009.stage.0037.task.0123 state: SCHEDULED
Update: #Update: pipeline.0009.stag

Update: Update: Update: pipeline.0006.stage.0025.task.0085 state: DONE
pipeline.0002.stage.0009.task.0031 state: SCHEDULED
pipeline.0002.stage.0009.task.0027 state: SUBMITTING
Update: pipeline.0002.stage.0009.task.0034 state: SCHEDULED
Update: Update: pipeline.0006.stage.0025.task.0086 state: DONE
pipeline.0002.stage.0009.task.0035 state: SCHEDULED
Update: Update: Update: pipeline.0002.stage.0009.task.0029 state: SCHEDULED
pipeline.0006.stage.0025.task.0087 state: DONE
pipeline.0002.stage.0009.task.0033 state: SUBMITTING
Update: pipeline.0002.stage.0009.task.0036 state: SCHEDULED
Update: Update: pipeline.0006.stage.0025.task.0079 state: DONE
pipeline.0004.stage.0017.task.0060 state: SCHEDULED
Update: Update: pipeline.0002.stage.0009.task.0030 state: SUBMITTING
Update: pipeline.0004.stage.0017.task.0061 state: SCHEDULED
pipeline.0006.stage.0025.task.0088 state: DONE
Update: pipeline.0004.stage.0017.task.0062 state: SCHEDULED
Update: pipeline.0006.stage.0025.task.0080 state: DONE
Update:

#Update: pipeline.0009.stage.0037.task.0121 state: DONE
#Update: Update: #pipeline.0005.stage.0021.task.0072 state: SUBMITTING
pipeline.0009.stage.0037.task.0122 state: DONE
Update: #pipeline.0009.stage.0037.task.0123 state: DONE
#Update: pipeline.0009.stage.0037.task.0124 state: DONE
#Update: Update: #pipeline.0009.stage.0037.task.0125 state: DONE
pipeline.0005.stage.0021.task.0073 state: SUBMITTING
Update: #pipeline.0009.stage.0037 state: DONE
#Update: pipeline.0005.stage.0021.task.0074 state: SUBMITTING
##Update: pipeline.0005.stage.0021.task.0075 state: SUBMITTING
##Update: pipeline.0005.stage.0021.task.0066 state: SUBMITTING
##Update: Update: pipeline.0005.stage.0021.task.0067 state: SUBMITTING
pipeline.0000.stage.0001.task.0004 state: DONE
#Update: #pipeline.0000.stage.0001.task.0005 state: DONE
Update: Update: #pipeline.0005.stage.0021.task.0068 state: SUBMITTING
pipeline.0000.stage.0001.task.0007 state: DONE
#Update: #pipeline.0000.stage.0001.task.0006 state: DONE
#Update: Upda

#Update: pipeline.0004.stage.0017.task.0059 state: DONE
#Update: #pipeline.0005.stage.0021.task.0071 state: DONE
#Update: pipeline.0005.stage.0021.task.0072 state: DONE
#Update: pipeline.0005.stage.0021.task.0073 state: DONE
#Update: pipeline.0005.stage.0021.task.0074 state: DONE
#Update: pipeline.0005.stage.0021.task.0075 state: DONE
#Update: pipeline.0005.stage.0021.task.0066 state: DONE
#Update: pipeline.0005.stage.0021.task.0068 state: DONE
#Update: pipeline.0005.stage.0021.task.0069 state: DONE
#Update: pipeline.0005.stage.0021.task.0070 state: DONE
####Update: #pipeline.0002.stage.0009.task.0036 state: DONE
Update: #pipeline.0002.stage.0009 state: DONE
#Update: pipeline.0004.stage.0017.task.0062 state: EXECUTED
#Update: pipeline.0004.stage.0017.task.0062 state: DONE
#Update: pipeline.0004.stage.0017.task.0055 state: EXECUTED
#Update: pipeline.0004.stage.0017.task.0055 state: DONE
#Update: pipeline.0004.stage.0017 state: DONE
#Update: pipeline.0005.stage.0021.task.0067 state: EXEC

#pipeline.0009.stage.0039 state: SCHEDULED
Update: pipeline.0004.stage.0018 state: DONE
Update: #pipeline.0004.stage.0018 state: SCHEDULED
Update: pipeline.0005.stage.0022.task.0076 state: DONE
Update: pipeline.0005.stage.0022 state: SCHEDULED
Update: pipeline.0005.stage.0022 state: DONE
###########Update: #pipeline.0006.stage.0027.task.0090 state: DONE
Update: pipeline.0006.stage.0027 state: DONE
Update: #pipeline.0006 state: DONE
###Update: pipeline.0000.stage.0003.task.0012 state: EXECUTED
#Update: pipeline.0000.stage.0003.task.0012 state: DONE
#Update: pipeline.0000.stage.0003 state: DONE
Update: pipeline.0000 state: DONE
##Update: pipeline.0002.stage.0011 state: SCHEDULING
Update: pipeline.0002.stage.0011.task.0038 state: SCHEDULING
#Update: pipeline.0004.stage.0019 state: SCHEDULING
#Update: pipeline.0004.stage.0019.task.0064 state: SCHEDULING
Update: pipeline.0005.stage.0023 state: SCHEDULING
#Update: pipeline.0005.stage.0023.task.0077 state: SCHEDULING
#Update: pipeline.0001.st

  0 --            21193520054.07
  1 --            32066349798.48
  2 --              145036328.12
  3 --              109036913.60
  4 --            57556250400.33
  5 --            48672732525.90
  6 --             1051010050.09
  7 --            14123294333.41
  8 --             4561763629.16
  9 --            10921976923.07
